In [1]:
import pandas as pd
import numpy as np
import datetime 
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from scipy import stats

In [2]:
# 데이터 불러오기
weather=pd.read_csv("risk_data/Weather_risk_table_final.csv", encoding='utf-8-sig')
birds=pd.read_csv("risk_data/birds_risk_table.csv", encoding='utf-8-sig')
distance=pd.read_csv("risk_data/distance_risk_final.csv", encoding='utf-8-sig')

In [3]:
weather.head()

,시도,month,day,avg_temp(C),daily_rain(mm),avg_r_humidity(%),temp_index,rain_index,humidity_index,risk
0,강원영동,1,1,1.2,0.0,33.2,0.0,0.0,0.0,0.0
1,강원영동,1,2,3.2,0.0,34.7,0.0,0.0,0.0,0.0
2,강원영동,1,3,3.4,0.0,35.5,0.0,0.0,0.0,0.0
3,강원영동,1,4,4.2,0.0,39.0,0.0,0.0,0.0,0.0
4,강원영동,1,5,3.2,1.8,43.8,0.0,0.0,0.0,0.0


In [4]:
birds.head()

,month,시도,발생건수(타겟),철새수,result,철새수/result,표준 risk,risk
0,1,강원도,2,626130,947733,66%,5,3
1,1,경기도,55,1242626,1072138,116%,5,5
2,1,경상남도,11,347682,968858,36%,5,1
3,1,경상북도,2,554884,947733,59%,5,2
4,1,광주광역시,0,49915,943038,5%,5,0


In [5]:
distance.head()

,month,dist_range,cnt,cumulative,risk
0,1,0,0,0,10
1,1,10,2,2,10
2,1,20,4,6,10
3,1,30,8,14,10
4,1,40,7,21,10


In [6]:
# 전국 시군 데이터 가져오기
all_sigun=pd.read_csv("전국_시군구(완).csv", encoding='utf-8-sig')

In [7]:
all_sigun.head()

,주소,x,y,xy
0,서울특별시 종로구,126.978990,37.573504,"126.978989954189,37.5735042429813"
1,서울특별시 중구,126.997555,37.563808,"126.997555182293,37.5638077703601"
2,서울특별시 용산구,126.990582,37.532431,"126.990582345331,37.5324310391314"
3,서울특별시 성동구,127.036930,37.563427,"127.036930141185,37.563427205337"
4,서울특별시 광진구,127.082385,37.538558,"127.082385189457,37.5385583136667"


In [8]:
all_sigun['시도']=np.nan
all_sigun['시군구']=np.nan
for i in range(len(all_sigun)):
    temp=all_sigun.loc[i,'주소'].split()
    all_sigun.loc[i,'시도']=temp[0]
    all_sigun.loc[i,'시군구']=temp[1]

In [9]:
all_sigun.head()

,주소,x,y,xy,시도,시군구
0,서울특별시 종로구,126.978990,37.573504,"126.978989954189,37.5735042429813",서울특별시,종로구
1,서울특별시 중구,126.997555,37.563808,"126.997555182293,37.5638077703601",서울특별시,중구
2,서울특별시 용산구,126.990582,37.532431,"126.990582345331,37.5324310391314",서울특별시,용산구
3,서울특별시 성동구,127.036930,37.563427,"127.036930141185,37.563427205337",서울특별시,성동구
4,서울특별시 광진구,127.082385,37.538558,"127.082385189457,37.5385583136667",서울특별시,광진구


In [10]:
yeongdong=['강릉시', '동해시', '속초시', '삼척시', '태백시', '고성군', '양양군']
yeongse=['원주시','춘천시','철원군','화천군','양구군','인제군','홍천군','평창군','정선군','영월군','횡성군']

In [11]:
print(set(weather['시도']))

{'강원영동', '전라남도', '경상북도', '경상남도', '제주특별자치도', '충청북도', '강원영서', '충청남도', '서울경기', '전라북도'}


In [12]:
print(set(all_sigun['시도']))

{'광주광역시', '강원도', '전라남도', '경상북도', '울산광역시', '경상남도', '제주특별자치도', '서울특별시', '충청북도', '인천광역시', '부산광역시', '충청남도', '세종특별자치시', '전라북도', '경기도', '대구광역시', '대전광역시'}


In [18]:
# 날씨 시도와 전국 시도 맞추기
w_dic={}
w_dic['인천광역시']='서울경기'
w_dic['대구광역시']='경상북도'
w_dic['부산광역시']='경상남도'
w_dic['울산광역시']='경상남도'
w_dic['서울특별시']='서울경기'
w_dic['대전광역시']='충청남도'
w_dic['경기도']='서울경기'
w_dic['광주광역시']='전라남도'
w_dic['세종특별자치시']='충청남도'
for sido in set(weather['시도']):
    if sido=='강원영동':
        for n in yeongdong:
            w_dic[n]=sido
    elif sido=='강원영서':
        for n in yeongse:
            w_dic[n]=sido
    else:
        w_dic[sido]=sido

In [19]:
print(w_dic)

{'인천광역시': '서울경기', '대구광역시': '경상북도', '부산광역시': '경상남도', '울산광역시': '경상남도', '서울특별시': '서울경기', '대전광역시': '충청남도', '경기도': '서울경기', '광주광역시': '전라남도', '세종특별자치시': '충청남도', '강릉시': '강원영동', '동해시': '강원영동', '속초시': '강원영동', '삼척시': '강원영동', '태백시': '강원영동', '고성군': '강원영동', '양양군': '강원영동', '전라남도': '전라남도', '경상북도': '경상북도', '경상남도': '경상남도', '제주특별자치도': '제주특별자치도', '충청북도': '충청북도', '원주시': '강원영서', '춘천시': '강원영서', '철원군': '강원영서', '화천군': '강원영서', '양구군': '강원영서', '인제군': '강원영서', '홍천군': '강원영서', '평창군': '강원영서', '정선군': '강원영서', '영월군': '강원영서', '횡성군': '강원영서', '충청남도': '충청남도', '서울경기': '서울경기', '전라북도': '전라북도'}


In [20]:
set(distance['month'])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

In [27]:
# 전체 데이터 만들기
cols=['month','day','시도','시군구','철새수','최초발생거리','평균온도','일강수량','상대습도','위험도']
total_df=pd.DataFrame(columns=cols)
total_index_df=pd.DataFrame(columns=cols)
total=[]
total_index=[]
for month in tqdm(range(1,13)):
    start_date='2020-'+str(month)+'-01'
    start_date=datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date=start_date+relativedelta(months=1)-datetime.timedelta(days=1)
    for day in tqdm(range(1,end_date.day+1)):
        for i in range(len(all_sigun)):
            q3=distance.query("month==@month").reset_index(drop=True)
            for j in range(len(q3)):
                temp={}
                index_temp={}
                temp['month']=month
                temp['day']=day
                sido=all_sigun.loc[i,'시도']
                temp['시도']=sido
                sigun=all_sigun.loc[i,'시군구']
                temp['시군구']=sigun
                index_temp['month']=month
                index_temp['day']=day
                index_temp['시도']=all_sigun.loc[i,'시도']
                index_temp['시군구']=all_sigun.loc[i,'시군구']

                q1=birds.query("month==@month and 시도==@sido").reset_index(drop=True)
                if len(q1)==0:
                    temp['철새수']=0
                    index_temp['철새수']=0
                else:
                    temp['철새수']=q1.loc[0,'철새수']
                    index_temp['철새수']=q1.loc[0,'risk']

                if temp['시도']=='강원도':
                    r_sigun=w_dic[sigun]
                    q2=weather.query("month==@month and day==@day and 시도==@r_sigun").reset_index(drop=True)
                else:
                    r_sido=w_dic[sido]
                    q2=weather.query("month==@month and day==@day and 시도==@r_sido").reset_index(drop=True)
                temp['평균온도']=q2.loc[0,'avg_temp(C)']
                temp['일강수량']=q2.loc[0,'daily_rain(mm)']
                temp['상대습도']=q2.loc[0,'avg_r_humidity(%)']
                index_temp['평균온도']=q2.loc[0,'temp_index']
                index_temp['일강수량']=q2.loc[0,'rain_index']
                index_temp['상대습도']=q2.loc[0,'humidity_index']

                temp['최초발생거리']=q3.loc[j,'dist_range']
                index_temp['최초발생거리']=q3.loc[j,'risk']
                total.append(temp)
                total_index.append(index_temp)

  3%|██▋                                                                                | 1/31 [00:23<11:33, 23.13s/it]


  7%|█████▌                                                                             | 2/30 [00:43<10:11, 21.84s/it]


100%|███████████████████████████████████████████████████████████████████████████████| 12/12 [2:21:56<00:00, 709.73s/it]


In [28]:
total_df=total_df.append(total)
total_index_df=total_index_df.append(total_index)

In [29]:
total_df.to_csv("Train_data_set.csv", encoding='utf-8-sig', index=False)

In [30]:
total_index_df.to_csv("Train_risk_data_set.csv", encoding='utf-8-sig', index=False)

In [7]:
total_df=pd.read_csv("Train_data_set.csv", encoding='utf-8-sig')

In [8]:
total_index_df=pd.read_csv("Train_risk_data_set.csv", encoding='utf-8-sig')

In [9]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4896378 entries, 0 to 4896377
Data columns (total 10 columns):
 #   Column  Dtype  
---  ------  -----  
 0   month   int64  
 1   day     int64  
 2   시도      object 
 3   시군구     object 
 4   철새수     int64  
 5   최초발생거리  int64  
 6   평균온도    float64
 7   일강수량    float64
 8   상대습도    float64
 9   위험도     float64
dtypes: float64(4), int64(4), object(2)
memory usage: 373.6+ MB


In [5]:
total_df.query("시군구=='종로구'").reset_index(drop=True)

,month,day,시도,시군구,철새수,최초발생거리,평균온도,일강수량,상대습도,위험도
0,1,1,서울특별시,종로구,126690,0,-1.8,0.0,64.2,NaN
1,1,1,서울특별시,종로구,126690,10,-1.8,0.0,64.2,NaN
2,1,1,서울특별시,종로구,126690,20,-1.8,0.0,64.2,NaN
3,1,1,서울특별시,종로구,126690,30,-1.8,0.0,64.2,NaN
4,1,1,서울특별시,종로구,126690,40,-1.8,0.0,64.2,NaN
...,...,...,...,...,...,...,...,...,...,...
23649,12,31,서울특별시,종로구,136044,240,-3.2,0.1,62.0,NaN
23650,12,31,서울특별시,종로구,136044,250,-3.2,0.1,62.0,NaN
23651,12,31,서울특별시,종로구,136044,260,-3.2,0.1,62.0,NaN
23652,12,31,서울특별시,종로구,136044,270,-3.2,0.1,62.0,NaN


In [10]:
# 상대습도 데이터는 다중공선성으로 독립변수로 지움
total_df.drop(['상대습도'], axis=1, inplace=True)
total_index_df.drop(['상대습도'], axis=1, inplace=True)

In [11]:
# 위험도 부여 (철새수: 0.3, 최초발생거리:0.5, 평균온도:0.14, 일강수량:0.06)
total_index_df['위험도']=total_index_df['철새수']*0.3+total_index_df['최초발생거리']*0.5+total_index_df['평균온도']*0.14+total_index_df['일강수량']*0.06
total_index_df['위험도']=total_index_df['위험도'].map(lambda x : np.round(x,1))

In [12]:
total_df['위험도']=total_index_df['위험도']

In [13]:
total_df.to_csv("Train_data_set.csv", encoding='utf-8-sig', index=False)

In [14]:
total_index_df.to_csv("Train_risk_data_set.csv", encoding='utf-8-sig', index=False)

### RandomForest을 이용한 데이터 평가

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
# pytorch로 괜찮은 성능 모델 확인 (XG BOOST, )
# 독립변수 계속 추가하기 
# 웹 UI (openstreet tool) (시각화 해보기) 근거 있는 주장 # 하나 입력했을때는 무조건 하는 것으로..

In [246]:
train_data=total_df.drop(['위험도'], axis=1)
target_data=total_df[['위험도']].astype('str')

In [247]:
print(set(total_df[['위험도']]['위험도'].values))

{0.9, 1.7, 2.0, 2.6, 4.5, 5.1, 4.8, 7.6, 7.0, 7.3, 2.3, 7.9, 5.4, 8.5, 6.0, 6.7, 3.5, 10.0, 9.4, 2.5, 4.0, 4.4, 4.6, 4.9, 0.6, 5.9, 5.0, 5.6, 6.5, 1.5, 6.9, 6.1, 0.0, 7.5, 7.1, 1.1, 7.4, 8.2, 8.8, 2.1, 3.6, 3.1, 1.2, 2.8, 0.3, 3.3, 0.8, 3.2, 3.8, 3.7, 4.2, 4.3, 1.8, 5.3, 5.8, 5.7, 6.3, 6.2, 6.8, 2.9, 2.4, 3.4}


In [258]:
# 시도, 시군구 범주화 하기
encoded_value, encoded_class = pd.factorize(train_data['시도'])
train_data['encoded_시도'] = encoded_value

In [259]:
print(set(encoded_value))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}


In [267]:
print(list(encoded_class))

['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도']


In [262]:
encoded_value2, encoded_class2 = pd.factorize(train_data['시군구'])
train_data['encoded_시군구'] = encoded_value2

In [265]:
print(set(encoded_value2))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206}


In [268]:
print(list(encoded_class2))

['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구', '서구', '동구', '영도구', '부산진구', '동래구', '남구', '북구', '해운대구', '사하구', '금정구', '연제구', '수영구', '사상구', '기장군', '수성구', '달서구', '달성군', '미추홀구', '연수구', '남동구', '부평구', '계양구', '강화군', '옹진군', '광산구', '유성구', '대덕구', '울주군', '세종시', '수원시', '성남시', '의정부시', '안양시', '부천시', '광명시', '평택시', '동두천시', '안산시', '고양시', '과천시', '구리시', '남양주시', '오산시', '시흥시', '군포시', '의왕시', '하남시', '용인시', '파주시', '이천시', '안성시', '김포시', '화성시', '광주시', '양주시', '포천시', '여주시', '연천군', '가평군', '양평군', '춘천시', '원주시', '강릉시', '동해시', '태백시', '속초시', '삼척시', '홍천군', '횡성군', '영월군', '평창군', '정선군', '철원군', '화천군', '양구군', '인제군', '고성군', '양양군', '청주시', '충주시', '제천시', '보은군', '옥천군', '영동군', '증평군', '진천군', '괴산군', '음성군', '단양군', '천안시', '공주시', '보령시', '아산시', '서산시', '논산시', '계룡시', '당진시', '금산군', '부여군', '서천군', '청양군', '홍성군', '예산군', '태안군', '전주시', '군산시', '익산시', '정읍시', '남원시', '김제시', '완주군', '진안군', '무주군', '장수군', '임실군', '순창군', '고창군', '

In [250]:
train_data.head()

,month,day,시도,시군구,철새수,최초발생거리,평균온도,상대습도,encoded_시도,encoded_시군구
0,1,1,서울특별시,종로구,126690,0,-1.8,64.2,0,0
1,1,1,서울특별시,종로구,126690,10,-1.8,64.2,0,0
2,1,1,서울특별시,종로구,126690,20,-1.8,64.2,0,0
3,1,1,서울특별시,종로구,126690,30,-1.8,64.2,0,0
4,1,1,서울특별시,종로구,126690,40,-1.8,64.2,0,0


In [251]:
train_data2=train_data.drop(['시도','시군구'], axis=1)

In [252]:
target_data2 = pd.get_dummies(target_data['위험도'])
target_data_result = pd.concat([target_data, target_data2], axis=1)

In [253]:
target_data_result.head()

,위험도,0.0,0.3,0.6,0.8,0.9,1.1,1.2,1.5,1.7,...,7.1,7.3,7.4,7.5,7.6,7.9,8.2,8.5,8.8,9.4
0,7.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [282]:
train_x, test_x, train_y, test_y = train_test_split(train_data2, target_data2, test_size = 0.2, random_state = 42)

In [169]:
clf = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=42)
clf.fit(train_x,train_y)

RandomForestClassifier(max_depth=10, n_estimators=10, random_state=42)

In [170]:
predict1 = clf.predict(test_x)
print(accuracy_score(test_y,predict1))

0.7570082387396403


In [178]:
test_result=pd.DataFrame(predict1, columns=target_data2.columns)

In [182]:
test_result.head()

,0.0,0.3,0.6,0.8,0.9,1.1,1.2,1.5,1.7,1.8,...,7.1,7.3,7.4,7.5,7.6,7.9,8.2,8.5,8.8,9.4
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [194]:
test_x.reset_index(drop=True, inplace=True)

In [195]:
test_x.head()

,month,day,시도,시군구,철새수,최초발생거리,평균온도,상대습도
0,5,6,3,11,0,10,16.6,61.0
1,1,14,15,72,3370301,100,-1.2,68.3
2,6,15,8,2,0,350,21.6,73.5
3,6,27,0,182,0,320,23.4,71.0
4,3,3,15,51,1180533,90,5.6,64.5


In [196]:
test_x['위험도']=np.nan

<ipython-input-196-4f4c3958aba2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x['위험도']=np.nan


In [197]:
test_result.head()

,0.0,0.3,0.6,0.8,0.9,1.1,1.2,1.5,1.7,1.8,...,7.1,7.3,7.4,7.5,7.6,7.9,8.2,8.5,8.8,9.4
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [204]:
# test 데이터 결합
df=test_result[test_result==1].stack().reset_index().drop(0,1)
df.set_index('level_0',inplace=True)

In [212]:
df.head()

,level_1
level_0,
0,7.0
1,7.5
2,2.0
3,0.0
4,6.0


In [206]:
test_x['위험도']=df['level_1']

<ipython-input-206-5ccffef85e2a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x['위험도']=df['level_1']


In [211]:
test_x.tail()

,month,day,시도,시군구,철새수,최초발생거리,평균온도,상대습도,위험도
489633,12,9,8,4,136044,230,0.8,57.3,2.0
489634,1,14,1,79,1242626,190,-1.7,58.3,3.5
489635,1,29,2,195,347682,170,2.4,68.3,NaN
489636,11,10,6,147,0,40,8.6,75.8,NaN
489637,3,21,1,136,1133953,170,7.7,63.3,NaN


In [241]:
test_x.to_csv("test_data(acc=75%).csv", encoding='utf-8-sig', index=False)

In [240]:
# 없는 데이터가 있음 (정확도에 미치지 못하는 것은 추출 못함)
for i in list(test_result.iloc[-1].index):
    if test_result.iloc[-1][i]==1:
        print(i)

In [ ]:
# 과거 발생했던 시점과 현재 모델 비교 가능
# 타겟데이터는 맨앞으로 빼는게 좋다. (뒤에 훈련데이터를 넣으면 되기 때문이다.)

### XGBoost Classifier 모델 이용

In [298]:
import xgboost

In [318]:
train_data=total_df.drop(['위험도'], axis=1)
target_data=total_df[['위험도']]

In [319]:
# 시도, 시군구 범주화 하기
encoded_value, encoded_class = pd.factorize(train_data['시도'])
train_data['encoded_시도'] = encoded_value

In [320]:
encoded_value2, encoded_class2 = pd.factorize(train_data['시군구'])
train_data['encoded_시군구'] = encoded_value2

In [321]:
train_data2=train_data.drop(['시도','시군구'], axis=1)

In [322]:
train_X, test_X, train_Y, test_Y = train_test_split(train_data2, target_data, test_size = 0.2, random_state = 42)

In [323]:
print(train_X.shape, test_X.shape)

(1958551, 8) (489638, 8)


In [324]:
print(train_Y.shape, test_Y.shape)

(1958551, 1) (489638, 1)


In [325]:
model = xgboost.XGBClassifier()
model.fit(train_X.values, train_Y.values)

[16:11:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [338]:
y_pred = model.predict(test_X.values)
predictions = [np.round(value,1) for value in y_pred]

In [345]:
len(predictions)

489638

In [352]:
len(list(test_Y['위험도']))

489638

In [371]:
test_Y['위험도'].values[0]

7.0

In [372]:
predictions[0]

7.0

In [376]:
print("(pred == y_test) score: {0:.3f}".format((y_pred==test_Y['위험도'].values).mean()))

(pred == y_test) score: 0.913


In [377]:
# 모델 저장
import joblib
joblib.dump(model, 'risk_data/xgboost.pkl')

['risk_data/xgboost.pkl']

In [378]:
# 모델 불러오기
loaded_model=joblib.load('risk_data/xgboost.pkl')

In [379]:
y_pred = loaded_model.predict(test_X.values)

In [380]:
print("(pred == y_test) score: {0:.3f}".format((y_pred==test_Y['위험도'].values).mean()))

(pred == y_test) score: 0.913
